# This code extracts audio from a video <br>and generates subtitles in SRT format

In [ ]:
import os
from moviepy.editor import VideoFileClip
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
import tempfile

def extract_audio(video_path, output_path):
    """Extract audio from video file"""
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(output_path)
    video.close()

def transcribe_audio_segment(audio_segment, recognizer):
    """Transcribource)
            try:
                text = recognizer.recognize_google(audio)
                return text
            except sr.UnknownValueError:
                return ""
            except sr.RequestError:
                return "[Error: Could not request results]"
            finally:
     e a single audio segment"""
    with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as temp_audio:
        audio_segment.export(temp_audio.name, format='wav')
        with sr.AudioFile(temp_audio.name) as source:
            audio = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio)
                return text
            except sr.UnknownValueError:
                return ""
            except sr.RequestError:
                return "[Error: Could not request results]"
            finally:
                os.unlink(temp_audio.name)

def generate_subtitles(video_path, output_srt_path, min_silence_len=500, silence_thresh=-40):
    """Generate subtitles from video and save as SRT file"""
    # Create temporary audio file
    temp_audio_path = 'temp_audio.wav'
    
    try:
        # Extract audio from video
        print("Extracting audio from video...")
        extract_audio(video_path, temp_audio_path)
        
        # Load audio file
        print("Processing audio...")
        audio = AudioSegment.from_wav(temp_audio_path)
        
        # Split audio on silence
        chunks = split_on_silence(
            audio,
            min_silence_len=min_silence_len,
            silence_thresh=silence_thresh
        )
        
        # Initialize speech recognizer
        recognizer = sr.Recognizer()
        
        # Create SRT file
        print("Generating subtitles...")
        with open(output_srt_path, 'w', encoding='utf-8') as srt_file:
            for i, chunk in enumerate(chunks, 1):
                # Calculate timestamps
                start_time = sum(len(c) for c in chunks[:i-1])
                end_time = start_time + len(chunk)
                
                # Format timestamps for SRT file
                start_timestamp = f"{start_time//60000:02d}:{(start_time//1000)%60:02d}:{start_time%1000:03d}"
                end_timestamp = f"{end_time//60000:02d}:{(end_time//1000)%60:02d}:{end_time%1000:03d}"
                
                # Transcribe audio chunk
                text = transcribe_audio_segment(chunk, recognizer)
                
                # Write to SRT file
                if text:
                    srt_file.write(f"{i}\n")
                    
                    srt_file.write(f"{start_timestamp} --> {end_timestamp}\n")
                    srt_file.write(f"{text}\n\n")
        
        print(f"Subtitles generated successfully! Saved to {output_srt_path}")
        
    finally:
        # Clean up temporary files
        if os.path.exists(temp_audio_path):
            os.remove(temp_audio_path)

# The call

In [2]:

video_path = '2024-11-02 19-57-47.mkv'
output_srt_path = video_path.rsplit('.', 1)[0] + '.srt'

generate_subtitles(video_path, output_srt_path)



Extracting audio from video...
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
Processing audio...
Generating subtitles...
Subtitles generated successfully! Saved to 2024-11-02 19-57-47.srt


# To extract the SRT file created<br>of the video

In [ ]:
import re
from typing import Dict
import pandas as pd
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def clean_srt(srt_path: str) -> str:
    """
    Clean and extract text content from SRT file
    """
    with open(srt_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Remove SRT indices and timestamps
    pattern = r'\d+\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n'
    cleaned_content = re.sub(pattern, '', content)
    
    # Remove empty lines and clean up spacing
    cleaned_content = '\n'.join(line.strip() for line in cleaned_content.split('\n') if line.strip())
    
    return cleaned_content

def create_analysis_prompt() -> PromptTemplate:
    """
    Create a detailed prompt template for video content analysis
    """
    template = """
    You are an expert content analyst. Analyze the following video transcript and answer these specific questions.
    Please provide detailed, factual responses based solely on the content provided.
    
    TRANSCRIPT:
    {text}
    
    Please analyze the content and answer the following questions:
    1. What is the main topic or theme of this video?
    2. Who are the key speakers or participants mentioned (if any)?
    3. What are the 3-5 most important points discussed?
    4. Are there any significant statistics or numerical data mentioned?
    5. What practical advice or recommendations are given (if any)?
    6. What are the key conclusions or takeaways?
    7. Are there any notable quotes or memorable statements?
    8. What technical terms or specialized vocabulary are used?
    9. What problems or challenges are discussed?
    10. What solutions or resolutions are proposed?

    Format your response as a Python dictionary with these exact keys:
    - main_topic
    - speakers
    - key_points
    - statistics
    - recommendations
    - conclusions
    - notable_quotes
    - technical_terms
    - challenges
    - solutions

    Each value should be either a string or a list of strings, as appropriate.
    Ensure the response is properly formatted for direct parsing into a Python dictionary.
    If any information is not available, use an empty string or empty list as appropriate.
    """
    
    return PromptTemplate(
        input_variables=["text"],
        template=template
    )

def analyze_content(api_key: str, content: str) -> Dict:
    """
    Analyze content using Gemini and return structured results
    """
    # Initialize Gemini LLM
    llm = GoogleGenerativeAI(
        model="models/gemini-1.5-flash",
        google_api_key=api_key,
        temperature=0.1
    )
    
    # Create and run the chain
    chain = LLMChain(
        llm=llm,
        prompt=create_analysis_prompt()
    )
    
    # Get response
    response = chain.run(text=content)
    
    # Convert string response to dictionary (safely)
    try:
        # Clean up the response string to ensure it's valid Python syntax
        response = response.replace("```python", "").replace("```", "").strip()
        result_dict = eval(response)
        
        # Ensure all expected keys are present
        expected_keys = [
            'main_topic', 'speakers', 'key_points', 'statistics',
            'recommendations', 'conclusions', 'notable_quotes',
            'technical_terms', 'challenges', 'solutions'
        ]
        
        for key in expected_keys:
            if key not in result_dict:
                result_dict[key] = [] if key in ['key_points', 'statistics', 'recommendations', 
                                               'notable_quotes', 'technical_terms', 'challenges', 
                                               'solutions'] else ""
        
        return result_dict
    
    except Exception as e:
        print(f"Error parsing LLM response: {e}")
        return {key: [] if key in ['key_points', 'statistics', 'recommendations', 
                                 'notable_quotes', 'technical_terms', 'challenges', 
                                 'solutions'] else "" for key in expected_keys}

def flatten_dict_for_pandas(d: Dict) -> Dict:
    """
    Flatten nested dictionary/list structures for pandas DataFrame
    """
    flattened = {}
    
    for key, value in d.items():
        if isinstance(value, list):
            # Join list items with '|' for better CSV readability
            flattened[key] = '|'.join(str(item) for item in value)
        else:
            flattened[key] = str(value)
    
    return flattened

# The call

In [ ]:
# The api key of gemini
api_key = 'YOUR_API_KEY'

# Get input file path
srt_path = '2024-11-02 19-57-47.srt'
output_path = 'document.csv'

# Process the content
print("Cleaning SRT content...")
cleaned_content = clean_srt(srt_path)

print("Analyzing content with Gemini...")
analysis_results = analyze_content(api_key, cleaned_content)

# Flatten the results for pandas
flattened_results = flatten_dict_for_pandas(analysis_results)

# Create DataFrame and save to CSV
df = pd.DataFrame([flattened_results])
df.to_csv(output_path, index=False)
print(f"Analysis results saved to {output_path}")
df.head()

Cleaning SRT content...
Analyzing content with Gemini...
Analysis results saved to document.csv


,main_topic,speakers,key_points,statistics,recommendations,conclusions,notable_quotes,technical_terms,challenges,solutions
0,Creating a desktop application to retrieve sub...,"The speaker is not explicitly identified, but ...",The application retrieves subtitles from a You...,,,The application is a simple but useful tool fo...,,PyWebView|requests|json|pyperclip|langchain|Ja...,,
